ParamNB is a small library that represents Parameters (defined using the [Param library](https://github.com/ioam/param)) graphically in a Jupyter notebook using [ipywidgets](https://ipywidgets.readthedocs.io).

To use ParamNB, first declare a Parameterized class with some Parameters:

In [ ]:
import param, paramnb

class Example(param.Parameterized):
    """An example parameterized class"""
    x = param.Parameter(default=3.14,doc="X position")
    y = param.Parameter(default="Not editable",constant=True)
    string_value = param.String(default="str",doc="A string")
    num_int = param.Integer(50000,bounds=(-200,100000))
    unbounded_int = param.Integer(23)
    float_with_hard_bounds = param.Number(8.2,bounds=(7.5,10))
    float_with_soft_bounds = param.Number(0.5,bounds=(0,None),softbounds=(0,2))
    unbounded_float = param.Number(30.01)    
    boolean = param.Boolean(True, doc="A sample Boolean parameter")
    select_string = param.ObjectSelector(default="yellow",objects=["red","yellow","green","blue"])
    select_fn = param.ObjectSelector(default=list,objects=[list,set,dict])
    int_list = paramnb.ListSelector(default=[3,5], objects=[1,3,5,7,9])
    single_file = paramnb.FileSelector(path='../*/*.py*')
    multiple_files = paramnb.MultiFileSelector(path='../*/*.py?')
    
Example.num_int

You can then display and edit these parameter values at the class level by wrapping the Parameterized class in a ``paramnb.Widgets`` object.  Because we want the user to have an opportunity to edit these values before the remainder of the notebook is executed, we also declare the ``execute`` mode to be `below`, which provide a `Run below` button to allow the user to execute the subsequent cells.

In [ ]:
def callback(pobj):
    print(pobj,Example.num_int,Example.unbounded_int)

paramnb.Widgets(Example,button=False,next_n=0)


If you now look at the values of the Parameters at the class level, you can see that they have changed to whatever was selected interactively above:

In [ ]:
Example.unbounded_int


In [ ]:
Example.num_int

In [ ]:
Example.print_param_defaults()

Subsequent cells can now refer to these values, e.g. as `Example.num_int`.  

There are several different options to tell `Widgets` how you want the selected values to affect subsequent cells:

- **Default**: Leave off the `execute` argument, and simply re-run the subsequent cells whenever you want them to use the newly changed values

- **`execute='below'`**: Provide a `Run below` button, which should be pressed after editing the parameter values when ready to run the subsequent cells.

- **`execute='next'`**: Provide a `Next` button, which should be pressed after editing the parameter values when ready to run the next cell.  Useful if you want to have a series of cells where the behavior depends on the settings chosen interactively in the previous cell.

- **`callback=<some_python_callable>`**: Allows you to define a callback that will be run whenever a parameter value changes (if `Widgets.on_change==True`) or otherwise whenever the `Next` or `Run below` buttons is pressed.